In [ ]:
import pandas as pd
import os, time
import numpy as np
from tensorflow.keras.applications import MobileNetV3Small
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#MobileNetv3Small
data_dir = "data/north_american_predators/original_downsampled_split/train"
dataset_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/original_downsampled_split/val"
dataset_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/original_downsampled_split/test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)
print(dataset_train.class_names)

Found 13823 files belonging to 8 classes.
Found 3454 files belonging to 8 classes.
Found 4317 files belonging to 8 classes.
['black_bear', 'brown_bear', 'cougar', 'coyote', 'human', 'non_human', 'polar_bear', 'wolf']


In [ ]:
def evaluate_model_tflite(interpreter, dataset):
    #List of dictionaries
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    correct = 0
    total = 0

    for images, labels in dataset:
        input_type = input_details[0]['dtype']
        # Convert the batch to the correct type
        images = images.numpy().astype(input_type)
        labels = labels.numpy()
        
        for i in range(images.shape[0]):
            test_image = np.expand_dims(images[i], axis=0)
            #From Tensorflow API Docs
            interpreter.set_tensor(input_details[0]['index'], test_image)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            #Gets the most prominent class
            pred = np.argmax(output)
            if pred == labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [4]:
base_model = tf.keras.applications.MobileNetV3Small(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
early_stop = EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

start_time = time.time()
model.fit(dataset_train, epochs=100, validation_data=dataset_val, callbacks=[early_stop])
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Epoch 1/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 20s 87ms/step - loss: 0.4434 - sparse_categorical_accuracy: 0.8741 - val_loss: 0.6257 - val_sparse_categorical_accuracy: 0.7823
Epoch 2/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step - loss: 0.1269 - sparse_categorical_accuracy: 0.9657 - val_loss: 0.6382 - val_sparse_categorical_accuracy: 0.8101
Epoch 3/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - loss: 0.0877 - sparse_categorical_accuracy: 0.9777 - val_loss: 0.6891 - val_sparse_categorical_accuracy: 0.8173
Epoch 4/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - loss: 0.0693 - sparse_categorical_accuracy: 0.9823 - val_loss: 0.6677 - val_sparse_categorical_accuracy: 0.8243
Epoch 5/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 16s 76ms/step - loss: 0.0576 - sparse_categorical_accuracy: 0.9858 - val_loss: 0.7286 - val_sparse_categorical_accuracy: 0.8208
Epoch 6/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step - loss: 0.0493 - sparse_categorical_accuracy: 0.9873 - val_loss: 0.6698 - val_sparse_categorical_acc

In [6]:
start_time = time.time()
test_loss, test_acc = model.evaluate(dataset_test, verbose=0)
end_time = time.time()
training_duration = end_time - start_time
print(f"Evaluation time: {training_duration:.2f} seconds")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Evaluation time: 4.44 seconds
Test Loss: 0.4186
Test Accuracy: 87.61%


In [7]:
model.save('model_MobileNetV3_original.keras')
print("Baseline model in Mb:", os.path.getsize(r'model_MobileNetV3_original.keras') / float(2**20))

Baseline model in Mb: 4.177478790283203


In [8]:
loaded_model = tf.keras.models.load_model("model_MobileNetV3_original.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_MobileNetV3_quantized_original.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp2hna8q8y\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp2hna8q8y\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmp2hna8q8y'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2410671639248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671638672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671638864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671639440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671639824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671639056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671640592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671640400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671638096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410671641168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410

In [9]:
loaded_model = tf.keras.models.load_model("model_MobileNetV3_original.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_MobileNetV3_original.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpaqmbelzo\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpaqmbelzo\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmpaqmbelzo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2410883376208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410883378896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2411962886352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410883380240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2410883377744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2411962890576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2411962890768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2411962890960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2411962891728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2411962890192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2411

In [ ]:
#MobileNetV3 trained on original images
interpreter = tf.lite.Interpreter(model_path="model_MobileNetV3_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_MobileNetV3_original.tflite') / float(2**20))

c:\Graduate_School\Fall_2025\ECE528\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


TFLite accuracy: 0.87607134584202
TFlite model in Mb: 3.576263427734375


In [ ]:
#MobileNetV3 quantized trained on original images
interpreter = tf.lite.Interpreter(model_path="model_MobileNetV3_quantized_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_MobileNetV3_quantized_original.tflite') / float(2**20))

TFLite accuracy: 0.861941162844568
TFlite model in Mb: 1.0618667602539062
